# Sequential Phenological Model

### Sequential Model Logic (best for D.C. Yoshino cherries)

1. **Chilling phase** (Nov 1 → until chilling requirement met):
   Count chilling units (typically hours ≤ 7°C / 45°F or a triangular function).
2. **Forcing phase** (starts the day after chilling requirement is met):
   Accumulate growing degree-days (GDD) base ~4–5°C until threshold → bloom.

#### Optimized Parameters for Washington, D.C. ( calibrated on 1921–2024 data)

In [3]:
# Best-fit parameters from recent studies

CHILLING_THRESHOLD = 1150      # chilling hours needed (can be 900–1400)
FORCING_THRESHOLD  = 3800      # degree-hours above 4.5°C needed
BASE_TEMP_CHILL    = 7.0       # °C
BASE_TEMP_FORCE    = 4.5       # °C
START_CHILLING     = "11-01"   # Nov 1 of previous year

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

daily_data = pd.read_csv("../data/USW00013743_reduced.csv")

def f_to_c(f):
    return (f - 32) * 5/9

def c_to_f(c):
    return c * 9/5 + 32

def sequential_model_daily(daily_df,
                          year_bloom,
                          chill_threshold=CHILLING_THRESHOLD,
                          force_threshold=FORCING_THRESHOLD,
                          t_base_chill=BASE_TEMP_CHILL,
                          t_base_force=BASE_TEMP_FORCE):
    """
    Returns predicted peak bloom DOY for a given bloom year.
    daily_df must contain full data from Nov (year-1) to May (bloom_year)
    """
    df = daily_df.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['tmean'] = (df['tmax'] + df['tmin']) / 2
    if df['tmax'].max() > 100:  # assume °F
        df['tmean_c'] = f_to_c(df['tmean'])
    else:
        df['tmean_c'] = df['tmean']

    # Period of interest
    start_chill = f"{year_bloom-1}-{START_CHILLING}"
    end_forcing = f"{year_bloom}-05-31"
    period = df[(df['date'] >= start_chill) & (df['date'] <= end_forcing)].copy()
    period = period.sort_values('date').reset_index(drop=True)

    # ------------------------------------------------------------------
    # Chilling units (simple Utah model style: 1 unit per hour ≤7°C)
    # ------------------------------------------------------------------
    period['chill_units'] = np.where(period['tmean_c'] <= t_base_chill, 1, 0) * 24  # hours/day

    period['cum_chill'] = period['chill_units'].cumsum()

    # Find dormancy release date (first day cum_chill exceeds threshold)
    dormancy_row = period[period['cum_chill'] >= chill_threshold].index
    if len(dormancy_row) == 0:
        return np.nan  # never met chilling (very rare)
    start_forcing_idx = dormancy_row[0]
    forcing_start_date = period.loc[start_forcing_idx, 'date']

    # ------------------------------------------------------------------
    # Forcing units (degree-hours above base)
    # ------------------------------------------------------------------
    forcing_data = period.iloc[start_forcing_idx:].copy()
    forcing_data['force_units'] = np.maximum(0, forcing_data['tmean_c'] - t_base_force) * 24

    forcing_data['cum_force'] = forcing_data['force_units'].cumsum()

    bloom_row = forcing_data[forcing_data['cum_force'] >= force_threshold].index
    if len(bloom_row) == 0:
        return np.nan
    bloom_date = forcing_data.loc[bloom_row[0], 'date']

    doy = bloom_date.dayofyear
    return doy, bloom_date.strftime('%Y-%m-%d'), forcing_start_date.strftime('%Y-%m-%d')

# ------------------------------------------------------------------
# Example: Run on all years 1921–1980 (or up to 2025)
# ------------------------------------------------------------------
results = []
for year in range(2020, 2026):   # or up to 2026
    pred = sequential_model_daily(daily_data, year_bloom=year)
    if pred[0] is not np.nan:
        results.append({'Year': year, 'Predicted_DOY': pred[0],
                        'Predicted_Date': pred[1], 'Dormancy_Release': pred[2]})

results_df = pd.DataFrame(results)
print(results_df)

# # Merge with your observed data
# full = your_observed_df.merge(results_df, on='Year', how='left')
# full['Error_days'] = full['bloom_day'] - full['Predicted_DOY']
# print("RMSE =", np.sqrt((full['Error_days']**2).mean()).round(2))

   Year  Predicted_DOY Predicted_Date Dormancy_Release
0  2020             72     2020-03-12       2020-01-26
1  2021             85     2021-03-26       2021-01-23
2  2022             75     2022-03-16       2022-01-30
3  2023             62     2023-03-03       2023-01-25
4  2024             73     2024-03-13       2024-02-03
5  2025             75     2025-03-16       2025-01-25
